In [1]:
### EER FACTORS CREATING

In [2]:
### INITIALIZATION

import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats as ss
import math     
    
%load_ext line_profiler

In [3]:
### GENERAL DATA PREPARATION

### Constants:
All = slice(None)
### Bloomberg structured data extraction parameters:
str_path_bb_hdf = 'Data_Files/Source_Files/Bloomberg_prepared.h5'
str_key_ret = 'bb_ret'
str_key_mmr = 'bb_mmr'
str_key_fx = 'bb_fx'
str_key_mcap = 'bb_mcap'
str_key_reer = 'bb_reer'
str_key_neer = 'bb_neer'
str_key_xcra = 'bb_xcra'
### Factors parameters:
str_date_start = '1996-08-01' # Start date to filter returns, market caps and factors
str_date_end = '2020-04-30' # End date to filter returns, market caps and factors
idx_date_range = pd.date_range(str_date_start, str_date_end, freq = 'BM')
list_ison = ['DM', 'EM', 'FM']
list_filter = ['DM', 'EM', 'FM']
list_truncate = [2.5, 2.0] # Standartization boundaries
bool_within_market = True # Standartization way
list_countries_to_exclude = ['VE'] # Countries not to play the game
flo_returns_similarity = 0.001
flo_returns_completeness = 1 / 3
int_concept_lag = 3 ### Lag in months for GDP  like concepts
int_concept_divider = 1000 # Divider to equalize concepts and GDP scales
int_regress_win = 60 # Regression window length for alternative sensitivity concept
int_regress_hl = 3 # Half-life period for alternative sensitivity concept
int_factor_addendum = 2.5 # list_truncate[0] # Factor scaler
dict_numer_ma_win = {} # Moving average window length for factors numerators
dict_numer_ma_win['LONG_TERM'] = round(260 / 12 * 3)
dict_numer_ma_win['SHORT_TERM'] = round(260 / 52 * 2)
dict_denom_ma_win = {} # Moving average window length for factors denomenators
dict_denom_ma_win['LONG_TERM'] = 260 * 5
dict_denom_ma_win['SHORT_TERM'] = round(260 / 12 * 6)
### Transitional results parameters:
str_path_trans_hdf = 'Data_Files/Test_Files/EER_factors_transitional.h5'
str_key_trans_ret = 'trans_ret'
str_key_trans_mcap = 'trans_mcap'
str_key_trans_factor = 'trans_factor'
### Measures parameters:
list_measure = ['fmb_weighted', 'qtl4'] # Efficacy measures list
list_back_period = [99, 10, 5] # Look back periods
int_horizon = 12 # Measure stats horizon
str_path_efficacy_hdf = 'Data_Files/Test_Files/EER_factors_stats.h5'
str_key_efficacy = 'fmb_weight'
str_path_efficacy_xlsx = 'Data_Files/Test_Files/EER_factors_stats.xlsx'
### Not used yet
int_lt_hl_months = 24
int_st_hl_months = 3

In [4]:
### DEFINING WEIGHTED AVERAGE

def weighted_average(ser_data, ser_weight = False):
    ### Default output:
    num_result = np.NaN
    ### Checking for data presence:
    if ser_data.count():       
        ### Checking for weights dataset:
        if isinstance(ser_weight, bool):
            ### Calculating of simple average:
            num_result = np.nanmean(ser_data.values)
        else:
            ### Weights filtering:
            list_weight = ser_weight[ser_data.dropna().index].values
            ### Checking for weights presence:
            if np.nansum(list_weight):
                ### Data filtering:
                list_data = ser_data.dropna().values
                ### Weighted average calculating:
                num_result = np.nansum(list_data * list_weight) / np.nansum(list_weight)
    ### Results output:
    return num_result

In [5]:
### DEFINING MULTI-STEP STANDARTIZATION FOR SEPARATE SERIES

def multistep_standartize(ser_data_source, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False):  
    ### Arrays of iterations properties:
    arr_mean = []
    arr_std = []
    ### Adding equal weights, when weights are absent:
    if isinstance(ser_weight, bool):
        ser_weight = pd.Series(1, index = ser_data_source.index)
        ser_weight.name = 'Weight'    
    ### Workhorse and resulting data vectors initialising:
    ser_data_iter = ser_data_source.dropna()
    ser_weight_iter = ser_weight.copy()
    ser_data_full = pd.Series(np.NaN, index = ser_data_iter.index)
    ### Looping by boundaries array:
    for num_bound_iter in arr_truncate:
        ### Properties calculating and saving:
        num_mean_iter = weighted_average(ser_data_iter, ser_weight_iter)
        num_std_iter = ser_data_iter.std()
        arr_mean.append(num_mean_iter)
        arr_std.append(num_std_iter)
        ser_data_iter = (ser_data_iter - num_mean_iter) / num_std_iter       
        ### Standartizing:
        if reuse_outliers:
            ser_data_iter[ser_data_iter.abs() >= num_bound_iter] = np.sign(ser_data_iter) * num_bound_iter 
        else:
            ### Saving to result and excluding from further calculations truncated values:             
            ser_data_full.where(ser_data_iter.abs() < num_bound_iter, np.sign(ser_data_iter) * num_bound_iter, inplace = True)
            ser_data_iter = ser_data_iter[ser_data_iter.abs() < num_bound_iter]           
    ### Aggregating result:
    if (reuse_outliers):
        ser_data_full = ser_data_iter
    else:     
        ser_data_full[ser_data_iter.index] = ser_data_iter
    ### Centering result:
    if (center_result):
        ser_result = ser_data_full - weighted_average(ser_data_full, ser_weight) 
    else:
        ser_result = ser_data_full    
    ### Result output:
    ser_result.name = str(ser_data_source.name) + '_standartized'
    if (full_result):
        return (ser_result, arr_mean, arr_std)
    else:
        return ser_result

In [6]:
### DEFINING MULTI-STEP STANDARTIZATION BY MARKET FOR SEPARATE SERIES

def ison_standartize(ser_to_manage, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False, within_market = False):
    ### Multi-step standartizing:
    if (within_market):
    ### Within market standartizing:
        ser_result = ser_to_manage.groupby(by = 'Market', group_keys = False).apply(multistep_standartize, arr_truncate, ser_weight, 
                                                                                    reuse_outliers, center_result, full_result)
    else:
    ### Full universe standartizing:
        ser_result = multistep_standartize(ser_to_manage, arr_truncate, ser_weight, reuse_outliers, center_result, full_result)
    ### Results output:
    return ser_result

In [7]:
### DEFINING MULTI-STEP STANDARTIZATION BY MARKET FOR FULL FACTOR STACK

def single_factor_standartize(ser_factor, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, within_market = False):
    ### Weights preparing:
    if isinstance(ser_weight, bool):
        ser_weight = pd.Series(1, index = ser_factor.index)
        ser_weight.name = 'Weight'    
    ### Multi-step standartizing:        
    df_factor = ser_factor.to_frame().join(ser_weight, how = 'left')
    df_factor.columns = ['Factor', 'Weight']
    ser_result = df_factor.groupby('Date', group_keys = False).apply\
    (lambda iter_df: ison_standartize(iter_df['Factor'], arr_truncate, iter_df['Weight'], reuse_outliers, center_result, False, within_market))
    ### Results output:
    ser_result.name = ser_factor.name
    return ser_result   

In [8]:
### DEFINING GROUP MULTI-STEP STANDARTIZATION BY MARKET FOR FULL FACTOR STACK FOR MULTIPLE FACTORS

def multi_factor_standartize(df_factor, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, within_market = False):
    
    dict_standartized = {}
    ### Single factor standartizing:
    for iter_factor in df_factor.columns:
        dict_standartized[iter_factor] = single_factor_standartize(df_factor[iter_factor], arr_truncate, ser_weight, 
                                                                   reuse_outliers, center_result, within_market)
    ### Concatenating to dataframe:
    df_result = pd.concat(dict_standartized, axis = 1)
    ### Results output:
    return df_result

In [9]:
### DEFINING EXPONENTIAL WEIGHT

def exp_weight_single(halflife_len = 3, num_element = 0):
    ### Weight calculating:
    num_period_factor = math.exp(math.log(0.5) / round(halflife_len))
    num_weight = np.exp(math.log(num_period_factor) * num_element)
    ### Result output:
    return num_weight

In [10]:
### DEFINING EFFICACY MEASURES FOR SINGLE FACTOR

def single_factor_multiple_efficacy_measures(ser_factor, ser_return, ser_weight, arr_measure, return_shift = 0, arr_truncate = [2.5, 2.0]):
    ### Declaring local constants & variables:
    All = slice(None)
    dict_measure = {}
    set_std_needed = {'fmb_std_eqw', 'fmb_std_weighted'}
    num_precision = 5 # For quintile bins rounding and borders controlling
    ### Defining date index shifting function:
    def date_reindex(iter_group, idx_date_range, return_shift = 0):
        ser_iter_result = iter_group.unstack('Country').reindex(idx_date_range).sort_index().shift(-return_shift).stack('Country', dropna = False).squeeze()
        ### Results output:
        return ser_iter_result    
    ### Defining MatLab style prctile function:
    def prctile(ser_to_perc, p):
        ### Sorted list preparing:
        list_to_perc = ser_to_perc.dropna().values
        list_sorted = np.sort(list_to_perc)
        ### Length calculating:
        num_len = len(list_to_perc)    
        ### Prctile calculating:
        num_result = np.interp(np.array(p), np.linspace(1 / (2 * num_len), (2 * num_len - 1) / (2 * num_len), num_len), list_sorted)
        ### Results output:
        return num_result
    ### Defining quintile bins distribution:
    def quintile_distribution(ser_iter_group, num_bins = 5, bool_right_included = True, bool_include_lowest = True, bool_populate_last = True):
        ### Bins preparing:
        list_bin = list(np.arange(0, 100, 100 / num_bins))[1 : ]
        list_bin = [round(iter_element / 100, 2) for iter_element in list_bin]
        list_bin = [prctile(ser_iter_group, iter_element) for iter_element in list_bin]
        list_bin = [ser_iter_group.min() - abs(ser_iter_group.min())] + list_bin + [ser_iter_group.max() + abs(ser_iter_group.max())]
        list_bin = [round(iter_element, num_precision) for iter_element in list_bin]
        if bool_populate_last:
            list_bin[-2] -= 10 ** (-num_precision)
        ### Bins distribution:
        ser_iter_distribution = pd.cut(ser_iter_group, bins = list_bin, labels = range(num_bins), 
                                       right = bool_right_included, include_lowest = bool_include_lowest, precision = num_precision)
        ### Results output:
        return ser_iter_distribution    
    ### Defining get_measure group level function:
    def get_measure(df_to_measure, iter_measure):
        ### Checking data sufficiency:
        if (len(df_to_measure.dropna().index) > 1):           
            ### Measure calculating:
            if (iter_measure == 'ic_spearman'):
                ### Spearmen information coefficient:
                list_factor = df_to_measure[['Factor', 'Return']].dropna()['Factor'].values
                list_return = df_to_measure[['Factor', 'Return']].dropna()['Return'].values
                num_result = ss.spearmanr(list_factor, list_return, nan_policy = 'omit').correlation
            if (iter_measure == 'ic_pearson'):
                ### Pearson information coefficient:
                list_factor = df_to_measure[['Factor', 'Return']].dropna()['Factor'].values
                list_return = df_to_measure[['Factor', 'Return']].dropna()['Return'].values                
                num_result = ss.pearsonr(list_factor, list_return)[0]
            if (iter_measure == 'fmb_eqw'):
                ### Fama-McBeth cross-sectional regression beta coefficient (equal weighted residuals):
                list_factor_added = df_to_measure[['Factor', 'Constant', 'Return']].dropna()[['Factor', 'Constant']].values
                list_return = df_to_measure[['Factor', 'Constant', 'Return']].dropna()['Return'].values
                wls_model = sm.OLS(endog = list_return, exog = list_factor_added, missing = 'drop', hasconst = False)
                wls_results = wls_model.fit()
                num_result = wls_results.params[0]
            if (iter_measure == 'fmb_weighted'):
                ### Fama-McBeth cross-sectional regression beta coefficient (market capitalization weighted residuals):
                list_factor_added = df_to_measure[['Factor', 'Constant', 'Return', 'Weight']].dropna()[['Factor', 'Constant']].values
                list_return = df_to_measure[['Factor', 'Constant', 'Return', 'Weight']].dropna()['Return'].values                
                list_weight = df_to_measure[['Factor', 'Constant', 'Return', 'Weight']].dropna()['Weight'].values
                wls_model = sm.WLS(endog = list_return, exog = list_factor_added, weights = pow(list_weight, 1/2), missing = 'drop', hasconst = False)
                wls_results = wls_model.fit()
                num_result = wls_results.params[0]
            if (iter_measure == 'fmb_std_eqw'):             
                ### Fama-McBeth cross-sectional regression beta coefficient (market capitalization weighted residuals):
                list_factor_std_added = df_to_measure[['Factor_std', 'Constant', 'Return']].dropna()[['Factor_std', 'Constant']].values
                list_return = df_to_measure[['Factor_std', 'Constant', 'Return']].dropna()['Return'].values                
                wls_model = sm.OLS(endog = list_return, exog = list_factor_std_added, missing = 'drop', hasconst = False)
                wls_results = wls_model.fit()
                num_result = wls_results.params[0]                 
            if (iter_measure == 'fmb_std_weighted'):
                ### Fama-McBeth cross-sectional regression beta coefficient (market capitalization weighted residuals):
                list_factor_std_added = df_to_measure[['Factor_std', 'Constant', 'Return', 'Weight']].dropna()[['Factor_std', 'Constant']].values
                list_return = df_to_measure[['Factor_std', 'Constant', 'Return', 'Weight']].dropna()['Return'].values                
                list_weight = df_to_measure[['Factor_std', 'Constant', 'Return', 'Weight']].dropna()['Weight'].values                
                wls_model = sm.WLS(endog = list_return, exog = list_factor_std_added, weights = pow(list_weight, 1/2), missing = 'drop', hasconst = False)
                wls_results = wls_model.fit()
                num_result = wls_results.params[0]  
            if (iter_measure == 'clp'):
                ### Constant leverage portfolio signed normalized multiplication sum:                
                ser_clp_weighted = df_to_measure[['Factor', 'Return', 'Weight']].dropna()['Factor']
                ser_clp_weighted = ser_clp_weighted * df_to_measure[['Factor', 'Return', 'Weight']].dropna()['Weight'].transform(np.sqrt)
                ser_clp_weighted.loc[ser_clp_weighted < 0] = -ser_clp_weighted / ser_clp_weighted[ser_clp_weighted < 0].sum()
                ser_clp_weighted.loc[ser_clp_weighted > 0] = ser_clp_weighted / ser_clp_weighted[ser_clp_weighted > 0].sum()
                num_result = (ser_clp_weighted * df_to_measure['Return']).sum()
            if ('qtl' in iter_measure):
                ### Interquntile range:
                num_bins = int(iter_measure.split('qtl')[1])   
                df_to_measure = df_to_measure[['Factor', 'Return', 'Constant']].dropna()
                df_to_measure['Return'] = df_to_measure['Return'] - df_to_measure['Return'].mean()
                df_to_measure['Factor'] = df_to_measure['Factor'].round(num_precision)
                ### Distribution factor values between quintile bins:
                ser_qtl_bins = quintile_distribution(df_to_measure['Factor'], num_bins, bool_right_included = True, bool_include_lowest = True, bool_populate_last = True)
                ser_qtl_bins.name = 'Bin'
                ### Mean return for each bin calculating:
                df_to_measure = df_to_measure.join(ser_qtl_bins)
                df_qtl_rets = df_to_measure.loc[(All), ['Return', 'Bin']]
                df_qtl_rets.set_index('Bin', append = True, inplace = True)
                ser_qtl_rets = df_qtl_rets.unstack('Bin').mean(axis = 0).droplevel(0).squeeze()
                num_result = ser_qtl_rets.iloc[-1] - ser_qtl_rets.iloc[0]                                 
        else:                          
            num_result = np.NaN
        ### Preparing results: 
        return num_result
    ### Preparing combined vectors for measures calculating:
    if (ser_weight.count() == 0):
        ser_weight = pd.Series(1, index = ser_factor.index)
    ### Region filter dropping:
    ser_factor = ser_factor.reset_index('Market', drop = True)
    ser_return = ser_return.reset_index('Market', drop = True)
    ser_weight = ser_weight.reset_index('Market', drop = True)
    ### Preparing shifted returns:
    idx_date_range = ser_return.index.get_level_values(0).unique()
    ser_return_shifted = ser_return.groupby('Country', group_keys = False).apply(date_reindex, idx_date_range, return_shift)
    ### Preparing combined vectors for measures calculating:
    df_to_measure = pd.concat([ser_factor, ser_return_shifted, ser_weight], axis = 1)
    df_to_measure.columns = ['Factor', 'Return', 'Weight']
    df_to_measure['Constant'] = 1
    if set_std_needed.intersection(set(arr_measure)):
        ser_factor_std = df_to_measure.dropna()['Factor'].groupby('Date').apply(ison_standartize, arr_truncate = arr_truncate, within_market = False)
        df_to_measure['Factor_std'] = ser_factor_std.reindex(df_to_measure.index)
    ### Looping efficacy measures for calculating measures timeseries:
    for iter_measure in arr_measure:
        dict_measure[iter_measure] = df_to_measure.groupby('Date').apply(get_measure, iter_measure = iter_measure)
    ### Preparing results:
    return pd.concat(dict_measure, axis = 1)

In [11]:
### DEFINING MEASURE STATISTICS CALCULATOR

def measure_stats(df_measures, arr_back_period = [99]):
    ### Declaring local constants & variables:
    dict_stats = {}
    ### Stats calculating:
    for iter_measure in df_measures.columns:
        dict_period = {}
        for iter_back_period in arr_back_period:
            ser_iter_measure = df_measures[iter_measure].dropna()
            idx_iter_range = pd.date_range(end = ser_iter_measure.index[-1], periods = iter_back_period * 12, freq = 'BM')
            ser_iter_measure = ser_iter_measure[idx_iter_range]            
            ser_iter_stats = pd.Series()
            ser_iter_stats['count'] = ser_iter_measure.count()
            ser_iter_stats['min'] = ser_iter_measure.min()
            ser_iter_stats['max'] = ser_iter_measure.max()        
            ser_iter_stats['mean'] = ser_iter_measure.mean()
            ser_iter_stats['std'] = ser_iter_measure.std()
            ser_iter_stats['median'] = ser_iter_measure.median()        
            ser_iter_stats['perc_25'] = ser_iter_measure.quantile(0.25, 'midpoint')
            ser_iter_stats['perc_75'] = ser_iter_measure.quantile(0.75, 'midpoint')
            ser_iter_stats['iq_range'] = ser_iter_measure.quantile(0.75, 'midpoint') - ser_iter_measure.quantile(0.25, 'midpoint')
            ser_iter_stats['mean_abs'] = ser_iter_measure.abs().mean()
            ser_iter_stats['t_stat'] = (ser_iter_measure.mean() / ser_iter_measure.std()) * np.sqrt(ser_iter_measure.count())  
            dict_period[iter_back_period] = ser_iter_stats
        dict_stats[iter_measure] = pd.concat(dict_period, axis = 1)
    ### Preparing results:
    return pd.concat(dict_stats, axis = 1)

In [12]:
### DEFINING SPECIAL CLP STATS

def special_clp_stats(ser_factor, ser_return, ser_weight, return_shift = 0):
    ### Declaring local constants & variables:    
    dict_clp_stats = {}
    list_bin_labels = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']
    ### Defining date index shifting function:
    def date_reindex(iter_group, idx_date_range, return_shift = 0):
        ser_iter_result = iter_group.unstack('Country').reindex(idx_date_range).sort_index().shift(-return_shift).stack('Country', dropna = False).squeeze()
        ### Results output:
        return ser_iter_result    
    ### Defining function for constant leverage portfolio normalizing:
    def get_normalized_clp(df_to_measure):
        ser_clp_weighted = df_to_measure['Factor'] * df_to_measure['Weight'].transform(np.sqrt)
        ### Checking data sufficiency:
        if (ser_clp_weighted.count() > 0):           
            ### Constant leverage portfolio signed normalized:
            ser_clp_weighted.loc[ser_clp_weighted < 0] = -ser_clp_weighted / ser_clp_weighted[ser_clp_weighted < 0].sum()
            ser_clp_weighted.loc[ser_clp_weighted > 0] = ser_clp_weighted / ser_clp_weighted[ser_clp_weighted > 0].sum()
            ser_result = ser_clp_weighted.copy()
        else:
            ser_result = pd.Series(np.NaN, index = ser_clp_weighted.index)
        ### Results output:
        return ser_result
    ### Defining function for constant leverage portfolio normalizing:
    def get_normalized_factor(df_to_measure):
        ser_clp_weighted = df_to_measure['Factor'] * df_to_measure['Weight'].transform(np.sqrt)
        ### Checking data sufficiency:
        if (ser_clp_weighted.count() > 0):           
            ### Factor signed normalized:
            ser_factor_normalized = df_to_measure['Factor']
            ser_factor_normalized.loc[ser_factor_normalized < 0] = -ser_factor_normalized / ser_factor_normalized[ser_factor_normalized < 0].sum()
            ser_factor_normalized.loc[ser_factor_normalized > 0] = ser_factor_normalized / ser_factor_normalized[ser_factor_normalized > 0].sum()
            ser_result = ser_factor_normalized
        else:
            ser_result = pd.Series(np.NaN, index = ser_clp_weighted.index) 
        ### Results output:
        return ser_result            
    ### Defining function for returns for constant leverage portfolio:
    def get_normalized_return(df_to_measure):
        ser_clp_weighted = df_to_measure['Factor'] * df_to_measure['Weight'].transform(np.sqrt)
        ### Checking data sufficiency:
        if (ser_clp_weighted.count() > 0):           
            ### Normalized return:  
            ser_result = df_to_measure['Return']
        else:
            ser_result = pd.Series(np.NaN, index = ser_clp_weighted.index)            
        ### Results output:
        return ser_result  
    ### Defining MatLab style prctile function:
    def prctile(ser_to_perc, p):
        ### Sorted list preparing:
        list_to_perc = ser_to_perc.dropna().values
        list_sorted = np.sort(list_to_perc)
        ### Length calculating:
        num_len = len(list_to_perc)    
        ### Prctile calculating:
        num_result = np.interp(np.array(p), np.linspace(1 / (2 * num_len), (2 * num_len - 1) / (2 * num_len), num_len), list_sorted)
        ### Results output:
        return num_result
    ### Defining quintile bins distribution:
    def quintile_distribution(ser_iter_group, num_bins = 5, bool_right_included = True, bool_include_lowest = True, bool_populate_last = True):
        ### Bins preparing:
        list_bin = list(np.arange(0, 100, 100 / num_bins))[1 : ]
        list_bin = [round(iter_element / 100, 2) for iter_element in list_bin]
        list_bin = [prctile(ser_iter_group, iter_element) for iter_element in list_bin]
        list_bin = [ser_iter_group.min() - abs(ser_iter_group.min())] + list_bin + [ser_iter_group.max() + abs(ser_iter_group.max())]
        list_bin = [round(iter_element, num_precision) for iter_element in list_bin]
        if bool_populate_last:
            list_bin[-2] -= 10 ** (-num_precision)
        ### Bins distribution:
        ser_iter_distribution = pd.cut(ser_iter_group, bins = list_bin, labels = range(num_bins), 
                                       right = bool_right_included, include_lowest = bool_include_lowest, precision = num_precision)
        ### Results output:
        return ser_iter_distribution 
    ### Preparing combined vectors for measures calculating:
    if (ser_weight.count() == 0):
        ser_weight = pd.Series(1, index = ser_factor.index)
    ### Region filter dropping:
    ser_factor = ser_factor.reset_index('Market', drop = True)
    ser_return = ser_return.reset_index('Market', drop = True)
    ser_weight = ser_weight.reset_index('Market', drop = True)      
    ### Preparing shifted returns:
    idx_date_range = ser_return.index.get_level_values(0).unique()
    ser_return_shifted = ser_return.groupby('Country', group_keys = False).apply(date_reindex, idx_date_range, return_shift)
    ### Preparing combined vectors for measures calculating:
    df_to_measure = pd.concat([ser_factor, ser_return_shifted, ser_weight], axis = 1)
    df_to_measure.columns = ['Factor', 'Return', 'Weight']
    ser_clp_normalized = df_to_measure.dropna().groupby('Date', group_keys = False).apply(get_normalized_clp)
    ser_factor_normalized = df_to_measure.dropna().groupby('Date', group_keys = False).apply(get_normalized_factor)
    ser_return_normalized = df_to_measure.dropna().groupby('Date', group_keys = False).apply(get_normalized_return)
    ### CLP stats calculating:
    dict_clp_stats['Average Bias'] = ser_factor_normalized.groupby('Country').mean()
#    dict_clp_stats['Temp'] = pd.Series(np.NaN, index = dict_clp_stats['Average Bias'].index)
    dict_clp_stats['Weights Sum'] = ser_clp_normalized.groupby('Country').sum()
    dict_clp_stats['Average Return'] = ser_return_normalized.groupby('Country').mean()   
    dict_clp_stats['Static Contribution'] = dict_clp_stats['Weights Sum'] * dict_clp_stats['Average Return']
    dict_clp_stats['Total Contribution'] = (ser_clp_normalized * ser_return_normalized).groupby('Country').sum()
    dict_clp_stats['Dynamic Contribution'] = dict_clp_stats['Total Contribution'] - dict_clp_stats['Static Contribution'] 
    ### CLP active weights calculating:
    ser_clp_delta = ser_clp_normalized.unstack('Date').stack('Date', dropna = False).swaplevel().sort_index(level = ['Date', 'Country'])
    ser_clp_delta = ser_clp_delta.fillna(0)
    num_clp_mean = ser_clp_delta.groupby('Country').mean().abs().sum()
    ser_clp_delta = ser_clp_delta.groupby('Country').apply(lambda iter_group: iter_group - iter_group.mean())
    num_clp_delta = (ser_clp_delta.abs().groupby('Date').sum() / (ser_clp_delta.abs().groupby('Date').sum() + num_clp_mean)).mean()
    df_clp_stats = pd.concat(dict_clp_stats, axis = 1).reindex(df_to_measure.index.get_level_values('Country').unique()).sort_index()
    ### Preparing sum:
    df_clp_sum = pd.DataFrame([[np.NaN, np.NaN, np.NaN, df_clp_stats['Static Contribution'].sum(), 
                               df_clp_stats['Total Contribution'].sum(), df_clp_stats['Dynamic Contribution'].sum()]], 
                              index = ['Sum'], columns = df_clp_stats.columns)
    ### Preparing expected based on active weights:
    df_clp_expected = pd.DataFrame([[np.NaN, np.NaN, np.NaN, 
                                     df_clp_stats['Total Contribution'].sum() * (1 - num_clp_delta), np.NaN, df_clp_stats['Total Contribution'].sum() * num_clp_delta]], 
                              index = ['Expected based on active weights =>'], columns = df_clp_stats.columns)    
    ### Adding totals:
    df_clp_stats = pd.concat([df_clp_stats, df_clp_sum, df_clp_expected], axis = 0, join = 'inner')
    ### CLP Bias calculating:   
    ser_clp_quintile = ser_clp_normalized.groupby('Date', group_keys = False).apply(quartile_distribution)
    df_clp_bias = ser_clp_quintile.to_frame()  
    df_clp_bias.columns = ['Bin']
    df_clp_bias['Quintile'] = 1
    df_clp_bias = df_clp_bias.set_index('Bin', append = True).unstack('Bin').fillna(0).droplevel(level = 0, axis = 1)
    df_clp_bias.columns = list(df_clp_bias.columns)  
    df_clp_bias = df_clp_bias[list_bin_labels]
    df_clp_bias = df_clp_bias.groupby('Country').mean()
    df_clp_bias.loc[:, 'Q5 - Q1'] = df_clp_bias['Q5'] - df_clp_bias['Q1']   
    df_clp_bias = df_clp_bias.reindex(df_to_measure.index.get_level_values('Country').unique()).sort_index()
    df_clp_bias = df_clp_bias
    ### Output results:
    return (df_clp_stats, df_clp_bias)

In [13]:
### DEFINING SPECIAL QTL STATS

def special_qtl_stats(ser_factor, ser_return, return_shift = 0, num_bins = 10):
    ### Declaring local constants & variables:
    All = slice(None)
    num_precision = 5 # For quintile bins rounding and borders controlling
    ### Defining date index shifting function:
    def date_reindex(iter_group, idx_date_range, return_shift = 0):
        ser_iter_result = iter_group.unstack('Country').reindex(idx_date_range).sort_index().shift(-return_shift).stack('Country', dropna = False).squeeze()
        ### Results output:
        return ser_iter_result    
    ### Defining MatLab style prctile function:
    def prctile(ser_to_perc, p):
        ### Sorted list preparing:
        list_to_perc = ser_to_perc.dropna().values
        list_sorted = np.sort(list_to_perc)
        ### Length calculating:
        num_len = len(list_to_perc)    
        ### Prctile calculating:
        num_result = np.interp(np.array(p), np.linspace(1 / (2 * num_len), (2 * num_len - 1) / (2 * num_len), num_len), list_sorted)
        ### Results output:
        return num_result
    ### Defining quintile bins distribution:
    def quintile_distribution(ser_iter_group, num_bins = 5, bool_right_included = True, bool_include_lowest = True, bool_populate_last = True):
        ### Bins preparing:
        list_bin = list(np.arange(0, 100, 100 / num_bins))[1 : ]
        list_bin = [round(iter_element / 100, 2) for iter_element in list_bin]
        list_bin = [prctile(ser_iter_group, iter_element) for iter_element in list_bin]
        list_bin = [ser_iter_group.min() - abs(ser_iter_group.min())] + list_bin + [ser_iter_group.max() + abs(ser_iter_group.max())]
        list_bin = [round(iter_element, num_precision) for iter_element in list_bin]
        if bool_populate_last:
            list_bin[-2] -= 10 ** (-num_precision)
        ### Bins distribution:
        ser_iter_distribution = pd.cut(ser_iter_group, bins = list_bin, labels = range(num_bins), 
                                       right = bool_right_included, include_lowest = bool_include_lowest, precision = num_precision)
        ### Results output:
        return ser_iter_distribution    

    ### Defining qts stats generator:
    def get_qtl_stats(df_to_measure):
        df_to_measure = df_to_measure[['Factor', 'Return', 'Constant']].dropna()
        if len(df_to_measure.index):
            df_to_measure['Return'] = df_to_measure['Return'] - df_to_measure['Return'].mean()
            df_to_measure['Factor'] = df_to_measure['Factor'].round(num_precision)
            ### Distribution factor values between quintile bins:
            ser_qtl_bins = quintile_distribution(df_to_measure['Factor'], num_bins, bool_right_included = True, bool_include_lowest = True, bool_populate_last = True)
            ser_qtl_bins.name = 'Bin'
            df_to_measure = df_to_measure.join(ser_qtl_bins)        
            ### Bins distribution:
            df_qtl_bins = df_to_measure.loc[(All), ['Constant', 'Bin']]
            df_qtl_bins.set_index('Bin', append = True, inplace = True)
            ser_qtl_bins = df_qtl_bins.unstack('Bin').sum(axis = 0).droplevel(0).squeeze()
            ### Return mean for each bin:
            df_qtl_rets = df_to_measure.loc[(All), ['Return', 'Bin']]
            df_qtl_rets.set_index('Bin', append = True, inplace = True)
            ser_qtl_rets = df_qtl_rets.unstack('Bin').mean(axis = 0).droplevel(0).squeeze()
        else:
            ser_qtl_bins = pd.Series(np.NaN, index = range(num_bins))
            ser_qtl_rets = pd.Series(np.NaN, index = range(num_bins))            
        ### Results output:
        ser_qtl_bins.name = 'Distribution'                
        ser_qtl_rets.name = 'Mean return'
        return pd.concat([ser_qtl_bins, ser_qtl_rets], axis = 0)      
        
    ### Region filter dropping:
    ser_factor = ser_factor.reset_index('Market', drop = True)
    ser_return = ser_return.reset_index('Market', drop = True)
    ### Preparing shifted returns:
    idx_date_range = ser_return.index.get_level_values(0).unique()
    ser_return_shifted = ser_return.groupby('Country', group_keys = False).apply(date_reindex, idx_date_range, return_shift)
    ### Preparing combined vectors for measures calculating:
    df_to_measure = pd.concat([ser_factor, ser_return_shifted], axis = 1)
    df_to_measure.columns = ['Factor', 'Return']
    df_to_measure['Constant'] = 1
    df_bins_stats = df_to_measure.groupby('Date', group_keys = False).apply(get_qtl_stats)
    ### Output results:
    df_bins_distribution = df_bins_stats.iloc[All, : num_bins]
    df_bins_return_mean = df_bins_stats.iloc[All, num_bins :]    
    return (df_bins_distribution, df_bins_return_mean)

In [14]:
### DEFINING SINGLE EFFICACY MEASURE FOR MULTIPLE FACTORS
    
def multiple_factor_single_efficacy_measure_stats(df_factors, ser_return, ser_weight, str_measure, num_back_period = 99, 
                                                  num_horizon = 12, list_region_xmo = ['DM', 'EM', 'FM']): 
    ### Declaring local constants & variables:
    All = slice(None)
    list_months = [1, 2, 3, 6, 9 ,12]
    list_range = [iter_month - 1 for iter_month in list_months if iter_month <= num_horizon]
    ### Defining full universe expanding for date:
    def universe_reindex(iter_group, idx_universe):
        df_iter_result = iter_group.unstack('Date').reindex(idx_universe).sort_index().stack('Date', dropna = False)
        ### Results output:
        return df_iter_result   
    ### Defining date index shifting function:
    def date_reindex(iter_group, idx_date_range, return_shift = 0):
        ser_iter_result = iter_group.unstack('Country').reindex(idx_date_range).sort_index().shift(-return_shift).stack('Country', dropna = False).squeeze()
        ### Results output:
        return ser_iter_result   
    ### Defining by date correrlation function:
    def corr_by_date(iter_group):
        num_iter_corr = iter_group['Corr_factor_minus'].corr(iter_group['Corr_factor_plus'])
        ### Results output:
        return num_iter_corr       
    ### Preparing expanded universe for autocorrelation performing:
    df_factors_region = df_factors.loc[(All, All, list_region_xmo), :]
    idx_date_range = df_factors_region.index.get_level_values(0).unique()
    idx_universe = df_factors_region.index.get_level_values(1).unique()
    df_factors_full = df_factors_region.reset_index('Market', drop = True).groupby('Date', group_keys = False).apply(universe_reindex, idx_universe).swaplevel()
    ### Factors looping:
    dict_factors_measures = {} ### Container for all factor stats
    dict_factors_autocorr = {} ### Container for autocorrelation results
    for iter_factor in df_factors.columns:
        ### Shifts looping for factors measures stats:
        ### Stats calculation:
        dict_factor_stats = {} ### Container for iterated factor stats
#        for iter_shift in range(num_horizon):
        for iter_shift in list_range:            
#            df_factor_filtered = df_factors[iter_factor].loc[All, All, list_region_xmo]
            df_factor_filtered = df_factors_region[iter_factor]
            df_iter_shift_measure = single_factor_multiple_efficacy_measures(df_factor_filtered, ser_return, ser_weight, [str_measure], iter_shift, list_truncate)
            df_iter_shift_stats = measure_stats(df_iter_shift_measure, [num_back_period])
            dict_factor_stats[iter_shift] = df_iter_shift_stats.loc[['mean', 't_stat'], (str_measure, num_back_period)]
        df_iter_factor_stats = pd.concat(dict_factor_stats, axis = 1)
        df_iter_factor_stats.columns = df_iter_factor_stats.columns + 1
        dict_factors_measures[iter_factor] = df_iter_factor_stats
        ### Autocorrelation calculation:
        ser_iter_factor = df_factors_full[iter_factor]
        ser_iter_factor_plus = ser_iter_factor.groupby('Country', group_keys = False).apply(lambda iter_group: iter_group.iloc[1 : ]).\
                                sort_index(level = ['Date', 'Country'])
        ser_iter_factor_minus = ser_iter_factor.groupby('Country', group_keys = False).apply(lambda iter_group: iter_group.iloc[: -1]).\
                                sort_index(level = ['Date', 'Country'])
        ### Artificial series combining for indexes synchronization:        
        ser_iter_factor_plus_shifted = ser_iter_factor_plus.groupby('Country', group_keys = False).apply(date_reindex, idx_date_range, 1)
        df_iter_factor_to_corr = pd.concat([ser_iter_factor_minus, ser_iter_factor_plus_shifted], axis = 1)
        df_iter_factor_to_corr.columns = ['Corr_factor_minus', 'Corr_factor_plus']
        dict_factors_autocorr[iter_factor] = pd.Series(df_iter_factor_to_corr.groupby('Date').apply(corr_by_date).mean(), index = ['autocorr'])
    ### Results output:
    df_factors_measures_stats = pd.concat(dict_factors_measures, axis = 0)
    df_factors_autocorr =  pd.concat(dict_factors_autocorr, axis = 1).transpose()
#    df_factors_coeff = df_factors_measures_stats.loc[(All, 'mean'), list_months].reset_index(1, drop = True)
    df_factors_coeff = df_factors_measures_stats.loc[(All, 'mean'), All].reset_index(1, drop = True)    
    df_factors_coeff.columns = [('coeff_' + str(iter_column)) for iter_column in df_factors_coeff.columns]
#    df_factors_t_stat = df_factors_measures_stats.loc[(All, 't_stat'), list_months].reset_index(1, drop = True)
    df_factors_t_stat = df_factors_measures_stats.loc[(All, 't_stat'), All].reset_index(1, drop = True)    
    df_factors_t_stat.columns = [('t_' + str(iter_column)) for iter_column in df_factors_t_stat.columns]
    df_factors_result = pd.concat([df_factors_autocorr, df_factors_coeff, df_factors_t_stat], axis = 1)    
    return (df_factors_result, df_factors_measures_stats)

In [15]:
### DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE

def ison_membership_loading(convert_to_daily = False):
    ### Reindexing function declaring:
    def reindex_month_ends(iter_group):
        iter_range = pd.date_range(iter_group.first_valid_index(), iter_group.last_valid_index(), freq = 'BM')
        iter_result = iter_group.reindex(iter_range)
        return iter_result    
    ### Declaring local constants & variables:
    path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path for membership source     
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Reindexing to show absent monthes for future daily resampling: 
    if (convert_to_daily):
        ser_market_membership = ser_market_membership.groupby('Country').apply(lambda iter_group: reindex_month_ends(iter_group.droplevel(1)))
        ser_market_membership.index.names = ['Country', 'Date']
        ser_market_membership = ser_market_membership.swaplevel()
        ser_market_membership = ser_market_membership.reset_index('Country').groupby('Country').resample('B').ffill().drop('Country', axis = 1).squeeze()
        ser_market_membership = ser_market_membership.swaplevel().sort_index(level = ['Country', 'Date'])
        
    return ser_market_membership

In [16]:
### MAIN SCRIPT: BLOOMBERG STRUCTURED DATA & ISON MEMBERSHIP EXTRACTION (NO PRELIMINARY DATA USING)

ser_returns = pd.read_hdf(str_path_bb_hdf, key = str_key_ret)
ser_mmr = pd.read_hdf(str_path_bb_hdf, key = str_key_mmr)
ser_fx_country = pd.read_hdf(str_path_bb_hdf, key = str_key_fx)
ser_mcap = pd.read_hdf(str_path_bb_hdf, key = str_key_mcap)
ser_reer = pd.read_hdf(str_path_bb_hdf, key = str_key_reer)
ser_neer = pd.read_hdf(str_path_bb_hdf, key = str_key_neer)
df_xcra_filled = pd.read_hdf(str_path_bb_hdf, key = str_key_xcra)
ser_ison = ison_membership_loading()

In [17]:
### TEMP

### List of countries with de-facto equal returns (to impact on hedged returns calculating)
#ser_ret_similarity_test = ser_returns.unstack('Currency').groupby('Country').apply(lambda df_country: (df_country['LOC'] - df_country['USD']).abs().mean())
#set_ret_usd_only = set(ser_ret_similarity_test.loc[ser_ret_similarity_test < flo_returns_similarity].index)
#
#ser_ret_similarity_test.loc[ser_ret_similarity_test < flo_returns_similarity].sort_values(ascending = False).sort_index()

#print(dict_ser_ret['LOC'].groupby('Country').apply(lambda ser_country: ser_country.count() / len(ser_country.index)).sort_values().head())
#print(dict_ser_ret['USD'].groupby('Country').apply(lambda ser_country: ser_country.count() / len(ser_country.index)).sort_values().head())
#print(dict_ser_ret['HEDGED'].groupby('Country').apply(lambda ser_country: ser_country.count() / len(ser_country.index)).sort_values().head())

#ser_ret_comleteness_test = ser_returns.groupby('Country').apply(lambda ser_country: ser_country.count() / len(ser_country.index))
#set_not_complete = set(ser_ret_comleteness_test.loc[ser_ret_comleteness_test < flo_returns_completeness].index)
#print(set_not_complete)
#ser_returns.loc[All, All, set_not_complete] = np.NaN
#sorted(ser_returns.index.get_level_values('Country').unique())

In [18]:
### MAIN SCRIPT: DATA PREPARING (NO PRELIMINARY DATA USING)

### List of countries with de-facto equal returns (to impact on hedged returns calculating)
ser_ret_similarity_test = ser_returns.unstack('Currency').groupby('Country').apply(lambda df_country: (df_country['LOC'] - df_country['USD']).abs().mean())
set_ret_usd_only = set(ser_ret_similarity_test.loc[ser_ret_similarity_test < flo_returns_similarity].index)
### List of countries with unsufficient data quantity:
ser_ret_comleteness_test = ser_returns.groupby('Country').apply(lambda ser_country: ser_country.count() / len(ser_country.index))
set_not_complete = set(ser_ret_comleteness_test.loc[ser_ret_comleteness_test < flo_returns_completeness].index)
### Filtering uncomplete countries:
ser_returns.loc[All, All, set_not_complete] = np.NaN
### Returns options preparing:
dict_ser_ret = {}
### Returns in local currency:
dict_ser_ret['LOC'] = ser_returns.loc['LOC', All, All].droplevel(0)
### Returns in USD:
dict_ser_ret['USD'] = ser_returns.loc['USD', All, All].droplevel(0)
### Hedged returns in local currency:
dict_ser_hedged = {}
### Filling data for countries with no MMR data:
set_ison_countries = set(dict_ser_ret['LOC'].index.get_level_values(1).unique())
set_mmr_countries = set(ser_mmr.index.get_level_values(1).unique())
set_no_mmr_countries = (set_ison_countries - set_mmr_countries) | set_ret_usd_only
set_to_hedge_countries = set_mmr_countries - set_no_mmr_countries
dict_ser_hedged['No_MMR'] = dict_ser_ret['LOC'].loc[All, set_no_mmr_countries]
### Money Market rates shifting forward:
ser_mmr_shifted = ser_mmr.groupby('Country').shift(1)
### Filling data for other countries:
df_ser_hedged = pd.DataFrame()
df_ser_hedged['Returns LOC'] = dict_ser_ret['LOC'].loc[All, set_to_hedge_countries]
df_ser_hedged = df_ser_hedged.join(ser_mmr_shifted, how = 'left')
df_ser_hedged.columns = ['Returns LOC', 'MMR LOC']
dict_ser_hedged['MMR_Based'] = df_ser_hedged.groupby('Country', group_keys = False)\
                               .apply(lambda df_country: (1 + df_country['Returns LOC']) * (1 + ser_mmr_shifted.loc[All, 'US'] / 12) / (1 + df_country['MMR LOC'] / 12) - 1)
#dict_ser_hedged['MMR_Based'] = df_ser_hedged.groupby('Country', group_keys = False)\
#                               .apply(lambda df_country: (1 + df_country['Returns LOC']) * (((1 + ser_mmr.loc[All, 'US']) / (1 + df_country['MMR LOC'])) ** (1 /12)) - 1)
### Aggregating hedged returns:
dict_ser_ret['HEDGED'] = pd.concat(dict_ser_hedged).droplevel(0).sort_index()

### Effective exchange rates options preparing:
dict_ser_eer = {}
dict_ser_eer['REER'] = ser_reer
dict_ser_eer['NEER'] = ser_neer

### Concepts options preparing:
dict_ser_concept = {}
### XCRA concept data shifting:
df_xcra_shifted = df_xcra_filled.groupby('Country').shift(int_concept_lag)
### XCRA concepts calculating:
dict_ser_concept['EXPIMP_GDP_rate'] = (df_xcra_shifted['Imports'] + df_xcra_shifted['Exports']) / df_xcra_shifted['GDP']
dict_ser_concept['EXP_GDP_rate'] = df_xcra_shifted['Exports'] / df_xcra_shifted['GDP']
dict_ser_concept['CA_GDP_rate'] = df_xcra_shifted['Current Account'] / df_xcra_shifted['GDP']
### XCRA concepts adjusting:
for iter_concept in dict_ser_concept:
    dict_ser_concept[iter_concept] = dict_ser_concept[iter_concept] / int_concept_divider
### XCRA concepts adjusting:
for iter_concept in dict_ser_concept:
    dict_ser_concept[iter_concept].loc[dict_ser_concept[iter_concept] <= -1] = -1 #-0.99
    dict_ser_concept[iter_concept] = np.log(1 + dict_ser_concept[iter_concept])    
### Neutral concept adding:
dict_ser_concept['NO_CONCEPT'] = pd.Series(1, index = dict_ser_concept['EXP_GDP_rate'].index)

### FX CONCEPT ADDING:

### Apriori weights for WLS calculation:
list_weights = []
for iter_num in range(int_regress_win):
    list_weights.append(exp_weight_single(int_regress_hl, iter_num))
list_weights = list_weights[::-1]
### Weighted least square performer defining:    
def perform_wls(ser_country_in):
    num_result = np.NaN
    ### Data retrieval from indexes:
    df_country_in = ser_country_in.reset_index(['Ret LOC', 'FX delta'])
    ### Weights adding:
    df_country_in['Weight'] = list_weights[-len(ser_country_in.index) : ]
    ### Columns renaming and NaN dropping:
    df_country_in.rename(columns = {0 : 'Constant'}, inplace = True)
    df_country_in.dropna(inplace = True)
    ### WLS performing:
    if (len(df_country_in.index) > 1):
        list_factor_added = df_country_in[['Ret LOC', 'Constant']].values
        list_basis = df_country_in['FX delta'].values
        list_weight = df_country_in['Weight'].values / df_country_in['Weight'].sum()
        wls_model = sm.WLS(endog = list_basis, exog = list_factor_added, weights = pow(list_weight, 1/2), missing = 'drop', hasconst = False)
        wls_results = wls_model.fit()
        ### Beta extracting:
        num_result = wls_results.params[0]
    ### Results output:
    return num_result
### Ordinary least square performer defining:    
def perform_ols(ser_country_in):
    num_result = np.NaN
    ### Data retrieval from indexes:
    df_country_in = ser_country_in.reset_index(['Ret LOC', 'FX delta'])
    ### Columns renaming and NaN dropping:
    df_country_in.rename(columns = {0 : 'Constant'}, inplace = True)
    df_country_in.dropna(inplace = True)
    ### WLS performing:
    if (len(df_country_in.index) > 1):
        list_factor_added = df_country_in[['Ret LOC', 'Constant']].values
        list_basis = df_country_in['FX delta'].values
        wls_model = sm.OLS(endog = list_basis, exog = list_factor_added, missing = 'drop', hasconst = False)
        wls_results = wls_model.fit()
        ### Beta extracting:
        num_result = wls_results.params[0]
    ### Results output:
    return num_result
### Local returns on FX rate regression beta data preparing:
ser_fx_delta = ser_fx_country.groupby('Country').transform(lambda ser_country: (ser_country / ser_country.shift(1) - 1))
df_sensitivity = ser_returns.loc['LOC', All, All].droplevel(0).to_frame().join(ser_fx_delta)
df_sensitivity.columns = ['Ret LOC', 'FX delta']
df_sensitivity['Constant'] = 1
### Fake indices creating for groupby -> rolling -> apply successfull performing:
ser_sensitivity = df_sensitivity.set_index(['Ret LOC', 'FX delta'], append = True).squeeze()
### Rolling WLS performing:
#df_beta = ser_sensitivity.groupby('Country', group_keys = False).rolling(int_regress_win, int_regress_win // 2).apply(perform_wls, raw = False)\
#                                                                                                               .reset_index(['Ret LOC', 'FX delta'])
df_beta = ser_sensitivity.groupby('Country', group_keys = False).rolling(int_regress_win, int_regress_win // 2).apply(perform_ols, raw = False)\
                                                                                                               .reset_index(['Ret LOC', 'FX delta'])
df_beta.rename(columns = {'Constant' : 'Beta'}, inplace = True)
### FX regression beta result preparing & inverting:
dict_ser_concept['FX_BETA'] = df_beta.drop(['Ret LOC', 'FX delta'], axis = 1).squeeze()    
dict_ser_concept['FX_BETA'] = -dict_ser_concept['FX_BETA']
### Concept series renaming:
for iter_concept in dict_ser_concept:
    dict_ser_concept[iter_concept].name = 'Multiplicator'

In [19]:
### MAIN SCRIPT: FACTORS CALCULATING (NO PRELIMINARY DATA USING)

### Containers for preliminary data:
dict_trans_ret_hdf = {}
dict_trans_mcap_hdf = {}
dict_trans_factor_hdf = {}
### Factors looping:
for iter_factor in ['LONG_TERM', 'SHORT_TERM']: # dict_numer_ma_win: # ['LONG_TERM'] : #
    ### Returns type looping:
    for iter_ret in dict_ser_ret: # ['HEDGED']: #
        ### Concept option looping:
        for iter_concept in ['EXP_GDP_rate', 'EXPIMP_GDP_rate', 'NO_CONCEPT', 'FX_BETA']: # ['EXP_GDP_rate']: # dict_ser_concept: # ['EXP_GDP_rate', 'NO_CONCEPT']: #
            ### Effective exchange rate type looping:
            for iter_eer in ['REER', 'NEER']: # ['REER']: # dict_ser_eer: #
                print(f'{iter_factor} / {iter_ret} / {iter_concept} / {iter_eer}')                
                ### Iteration data loading:
                ser_iter_ret = dict_ser_ret[iter_ret]
                ser_iter_concept = dict_ser_concept[iter_concept]
                ser_iter_eer = dict_ser_eer[iter_eer]
                int_iter_numer = dict_numer_ma_win[iter_factor]
                int_iter_denom = dict_denom_ma_win[iter_factor]                
                ### Factor calculating and ISONing:
                ser_iter_factor = -ser_iter_eer.groupby('Country').transform(lambda ser_country: np.log(ser_country.rolling(int_iter_numer, int_iter_numer // 2).mean() / \
                                                                                                        ser_country.rolling(int_iter_denom, int_iter_denom // 2).mean()))
                ser_iter_factor = ser_iter_factor.to_frame().join(ser_ison, how = 'left').set_index('Market', append = True).squeeze()
                ser_iter_factor.name = 'Factor'
                ### Returns shifting and ISONing:
                ser_iter_ret = ser_iter_ret.groupby('Country').shift(periods = -1).to_frame().join(ser_ison, how = 'left').set_index('Market', append = True).squeeze()
                ### Concept multiplicator ISONing:
                ser_iter_concept = ser_iter_concept.to_frame().join(ser_ison, how = 'left').set_index('Market', append = True).squeeze()
                ### Regions clearing:
                ser_iter_ret = ser_iter_ret.loc[idx_date_range, All, list_ison]
                ser_iter_mcap = ser_mcap.loc[idx_date_range, All, list_ison]
                ser_iter_factor = ser_iter_factor.loc[idx_date_range, All, list_ison]
                ser_iter_concept = ser_iter_concept.loc[idx_date_range, All, list_ison]
                ### Countries filtering:
                ser_iter_ret = ser_iter_ret.drop(list_countries_to_exclude, level = 'Country')
                ser_iter_mcap = ser_iter_mcap.drop(list_countries_to_exclude, level = 'Country')
                ser_iter_factor = ser_iter_factor.drop(list_countries_to_exclude, level = 'Country') 
                ser_iter_concept = ser_iter_concept.drop(list_countries_to_exclude, level = 'Country') 
                ### Factor and Multiplicator standratizing (Multiplicator shifting), multiplying and restandartizing:
                ser_iter_factor_std = single_factor_standartize(ser_iter_factor, list_truncate, within_market = bool_within_market)
                if (iter_concept != 'NO_CONCEPT'):
                    ser_iter_concept_std = single_factor_standartize(ser_iter_concept, list_truncate, within_market = bool_within_market)
                    ser_iter_factor_std = ser_iter_factor_std * (ser_iter_concept_std + int_factor_addendum).abs()
                    ser_iter_factor_std = single_factor_standartize(ser_iter_factor_std, list_truncate, within_market = bool_within_market)                    
                ### Preliminary results saving:
                str_iter_key = '__'.join([iter_factor, iter_ret, iter_concept, iter_eer])
                dict_trans_ret_hdf[str_iter_key] = ser_iter_ret
                dict_trans_mcap_hdf[str_iter_key] = ser_iter_mcap
                dict_trans_factor_hdf[str_iter_key] = ser_iter_factor_std                     

LONG_TERM / LOC / EXP_GDP_rate / REER
LONG_TERM / LOC / EXP_GDP_rate / NEER
LONG_TERM / LOC / EXPIMP_GDP_rate / REER
LONG_TERM / LOC / EXPIMP_GDP_rate / NEER
LONG_TERM / LOC / NO_CONCEPT / REER
LONG_TERM / LOC / NO_CONCEPT / NEER
LONG_TERM / LOC / FX_BETA / REER
LONG_TERM / LOC / FX_BETA / NEER
LONG_TERM / USD / EXP_GDP_rate / REER
LONG_TERM / USD / EXP_GDP_rate / NEER
LONG_TERM / USD / EXPIMP_GDP_rate / REER
LONG_TERM / USD / EXPIMP_GDP_rate / NEER
LONG_TERM / USD / NO_CONCEPT / REER
LONG_TERM / USD / NO_CONCEPT / NEER
LONG_TERM / USD / FX_BETA / REER
LONG_TERM / USD / FX_BETA / NEER
LONG_TERM / HEDGED / EXP_GDP_rate / REER
LONG_TERM / HEDGED / EXP_GDP_rate / NEER
LONG_TERM / HEDGED / EXPIMP_GDP_rate / REER
LONG_TERM / HEDGED / EXPIMP_GDP_rate / NEER
LONG_TERM / HEDGED / NO_CONCEPT / REER
LONG_TERM / HEDGED / NO_CONCEPT / NEER
LONG_TERM / HEDGED / FX_BETA / REER
LONG_TERM / HEDGED / FX_BETA / NEER
SHORT_TERM / LOC / EXP_GDP_rate / REER
SHORT_TERM / LOC / EXP_GDP_rate / NEER
SHORT_TERM

In [20]:
### MAIN SCRIPT: SAVING TRANSITIONAL RESULTS (NO PRELIMINARY DATA USING)

for iter_key in dict_trans_ret_hdf:
    dict_trans_ret_hdf[iter_key].to_hdf(str_path_trans_hdf, key = str_key_trans_ret + '__' + iter_key, mode = 'a')
    dict_trans_mcap_hdf[iter_key].to_hdf(str_path_trans_hdf, key = str_key_trans_mcap + '__' + iter_key, mode = 'a')
    dict_trans_factor_hdf[iter_key].to_hdf(str_path_trans_hdf, key = str_key_trans_factor + '__' + iter_key, mode = 'a')     

In [20]:
### FACTORS PERFORMING (PRELIMINARY DATA USED)

### Results container:
dict_measure_stats = {}
### Factors looping:
for iter_factor in ['LONG_TERM', 'SHORT_TERM']: # dict_numer_ma_win: # ['LONG_TERM'] : #
    ### Returns type looping:
    for iter_ret in dict_ser_ret: # ['HEDGED']: #
        ### Concept option looping:
        for iter_concept in ['EXP_GDP_rate', 'EXPIMP_GDP_rate', 'NO_CONCEPT', 'FX_BETA']: # ['EXP_GDP_rate']: # dict_ser_concept: # ['EXP_GDP_rate', 'NO_CONCEPT']: #
            ### Effective exchange rate type looping:
            for iter_eer in ['REER', 'NEER']: # ['REER']: # dict_ser_eer: #
                ### Transitional data loading, filtering and concatenating:
                str_iter_key = '__'.join([iter_factor, iter_ret, iter_concept, iter_eer])  
                print(str_iter_key)
                ser_iter_ret = pd.read_hdf(str_path_trans_hdf, key = str_key_trans_ret + '__' + str_iter_key).loc[All, All, list_filter]
                ser_iter_mcap = pd.read_hdf(str_path_trans_hdf, key = str_key_trans_mcap + '__' + str_iter_key).loc[All, All, list_filter]
                ser_iter_factor = pd.read_hdf(str_path_trans_hdf, key = str_key_trans_factor + '__' + str_iter_key).loc[All, All, list_filter]
                ser_iter_factor.name = str_iter_key
                ### Measure calculating:
                dict_measure_stats[str_iter_key] = multiple_factor_single_efficacy_measure_stats(ser_iter_factor.to_frame(), ser_iter_ret * 100, ser_iter_mcap,
                                                                                                 list_measure[0], list_back_period[0], int_horizon)[0]
### Results preparing:
df_factor_measure_stats = pd.concat(dict_measure_stats).droplevel(0)
df_factor_measure_stats.index.name = 'KEY'
df_factor_measure_stats.reset_index(inplace = True)
df_factor_measure_stats['Factor'] = df_factor_measure_stats['KEY'].str.split('__').str[0]
df_factor_measure_stats['Returns'] = df_factor_measure_stats['KEY'].str.split('__').str[1]
df_factor_measure_stats['Multiplicator'] = df_factor_measure_stats['KEY'].str.split('__').str[2]
df_factor_measure_stats['EER'] = df_factor_measure_stats['KEY'].str.split('__').str[3]
df_factor_measure_stats = df_factor_measure_stats.set_index(['Factor', 'Returns', 'Multiplicator', 'EER']).drop('KEY', axis = 1)
df_factor_measure_stats.to_hdf(str_path_efficacy_hdf, key = str_key_efficacy, mode = 'w')

LONG_TERM__LOC__EXP_GDP_rate__REER
LONG_TERM__LOC__EXP_GDP_rate__NEER
LONG_TERM__LOC__EXPIMP_GDP_rate__REER
LONG_TERM__LOC__EXPIMP_GDP_rate__NEER
LONG_TERM__LOC__NO_CONCEPT__REER
LONG_TERM__LOC__NO_CONCEPT__NEER
LONG_TERM__LOC__FX_BETA__REER
LONG_TERM__LOC__FX_BETA__NEER
LONG_TERM__USD__EXP_GDP_rate__REER
LONG_TERM__USD__EXP_GDP_rate__NEER
LONG_TERM__USD__EXPIMP_GDP_rate__REER
LONG_TERM__USD__EXPIMP_GDP_rate__NEER
LONG_TERM__USD__NO_CONCEPT__REER
LONG_TERM__USD__NO_CONCEPT__NEER
LONG_TERM__USD__FX_BETA__REER
LONG_TERM__USD__FX_BETA__NEER
LONG_TERM__HEDGED__EXP_GDP_rate__REER
LONG_TERM__HEDGED__EXP_GDP_rate__NEER
LONG_TERM__HEDGED__EXPIMP_GDP_rate__REER
LONG_TERM__HEDGED__EXPIMP_GDP_rate__NEER
LONG_TERM__HEDGED__NO_CONCEPT__REER
LONG_TERM__HEDGED__NO_CONCEPT__NEER
LONG_TERM__HEDGED__FX_BETA__REER
LONG_TERM__HEDGED__FX_BETA__NEER
SHORT_TERM__LOC__EXP_GDP_rate__REER
SHORT_TERM__LOC__EXP_GDP_rate__NEER
SHORT_TERM__LOC__EXPIMP_GDP_rate__REER
SHORT_TERM__LOC__EXPIMP_GDP_rate__NEER
SHORT_TERM

In [21]:
### FACTORS EXTRACTING (PRELIMINARY DATA USED)

df_factor_measure_stats = pd.read_hdf(str_path_efficacy_hdf, key = str_key_efficacy)
df_factor_measure_stats.to_excel(str_path_efficacy_xlsx, merge_cells = False)

In [22]:
### TESTING

iter_factor = 'LONG_TERM' # 'SHORT_TERM' #
iter_ret = 'HEDGED' # 'LOC' # 'USD' # 'HEDGED' #
iter_concept = 'EXP_GDP_rate' # 'NO_CONCEPT' #
iter_eer = 'REER' # 'NEER' #
### Transitional data loading, filtering and concatenating:
str_iter_key = '__'.join([iter_factor, iter_ret, iter_concept, iter_eer])  
print(str_iter_key)
ser_iter_ret = pd.read_hdf(str_path_trans_hdf, key = str_key_trans_ret + '__' + str_iter_key).loc[All, All, list_filter]
ser_iter_mcap = pd.read_hdf(str_path_trans_hdf, key = str_key_trans_mcap + '__' + str_iter_key).loc[All, All, list_filter]
ser_iter_factor = pd.read_hdf(str_path_trans_hdf, key = str_key_trans_factor + '__' + str_iter_key).loc[All, All, list_filter]
ser_iter_ret.to_excel('Data_Files/Test_Files/Example_EER_Returns.xlsx', merge_cells = False)
ser_iter_mcap.to_excel('Data_Files/Test_Files/Example_EER_Market_Caps.xlsx', merge_cells = False)
ser_iter_factor.to_excel('Data_Files/Test_Files/Example_EER_Factor.xlsx', merge_cells = False)

LONG_TERM__HEDGED__EXP_GDP_rate__REER


In [31]:
### TEMP

ser_iter_factor.name = str_iter_key
df_test_res = single_factor_multiple_efficacy_measures(ser_iter_factor, ser_iter_ret * 100, ser_iter_mcap, list_measure)
df_test_stats = measure_stats(df_test_res, list_back_period)
df_bins_distribution, df_bins_return_mean = special_qtl_stats(ser_iter_factor, ser_iter_ret * 100, num_bins = 4)
df_bins_return_mean.to_excel('Data_Files/Test_Files/QTL_debugging.xlsx', merge_cells = False)
df_test_stats